In [8]:
import itk
from itk import TubeTK as tube

from itkwidgets import view

import numpy as np

In [9]:
InputFilename = "../data/results/CT-Lungs.mha"

imLungs = itk.imread(InputFilename, itk.F)

In [15]:
ImageType = itk.Image[itk.F, 3]

imMath = tube.ImageMath.New(imLungs)
imMath.Blur(3)
imBlur = imMath.GetOutput()
imBlurArray = itk.GetArrayViewFromImage(imBlur)

numSeeds = 15
seedCoverage = 20
seedCoord = np.zeros([numSeeds,3])
for i in range(numSeeds):
    seedCoord[i] = np.unravel_index(np.argmax(imBlurArray, axis=None), imBlurArray.shape)
    indx = [int(seedCoord[i][0]),int(seedCoord[i][1]),int(seedCoord[i][2])]
    minX = max(indx[0]-seedCoverage,0)
    maxX = max(indx[0]+seedCoverage,imBlurArray.shape[0])
    minY = max(indx[1]-seedCoverage,0)
    maxY = max(indx[1]+seedCoverage,imBlurArray.shape[1])
    minZ = max(indx[2]-seedCoverage,0)
    maxZ = max(indx[2]+seedCoverage,imBlurArray.shape[2])
    imBlurArray[minX:maxX,minY:maxY,minZ:maxZ]=-1024
    indx.reverse()
    seedCoord[:][i] = imLungs.TransformIndexToPhysicalPoint(indx)
print(seedCoord)

[[  62.91250461 -152.2656281    73.12804806]
 [ -64.3656224  -183.8968786   143.16867417]
 [ -71.14374751 -171.84687841  122.83429885]
 [ -59.09374732 -137.95625287   58.81867284]
 [ -77.1687476  -159.79687822   63.33742291]
 [  71.19687974 -143.98125297   43.00304759]
 [ -68.88437247 -131.17812777   43.00304759]
 [-118.59062326 -136.45000285  146.18117422]
 [ -99.00937295 -115.36250252  158.23117441]
 [ -97.50312293 -189.9218787    81.41242319]
 [ -89.97187281 -147.74687803   46.76867265]
 [ -47.79687214 -126.65937769   27.18742234]
 [ -74.15624756 -110.09062743  140.90929914]
 [ -86.20624775 -135.69687784   28.69367236]
 [-105.78749806 -159.04375321   72.37492305]]


In [20]:
# Manually extract a few vessels to form an image-specific training set
vSeg = ttk.SegmentTubes.New(Input=imLungs)
vSeg.SetVerbose(True)
vSeg.SetMinRoundness(0.3)
vSeg.SetMinCurvature(0.002)
vSeg.SetRadiusInObjectSpace( 0.5 )
for i in range(numSeeds):
    print("**** Processing seed " + str(i) + " : " + str(seedCoord[i]))
    vSeg.ExtractTubeInObjectSpace( seedCoord[i], i )
    
tubeMaskImage = vSeg.GetTubeMaskImage()

**** Processing seed 0 : [  62.91250461 -152.2656281    73.12804806]
**** Processing seed 1 : [ -64.3656224  -183.8968786   143.16867417]
**** Processing seed 2 : [ -71.14374751 -171.84687841  122.83429885]
**** Processing seed 3 : [ -59.09374732 -137.95625287   58.81867284]
**** Processing seed 4 : [ -77.1687476  -159.79687822   63.33742291]
**** Processing seed 5 : [  71.19687974 -143.98125297   43.00304759]
**** Processing seed 6 : [ -68.88437247 -131.17812777   43.00304759]
**** Processing seed 7 : [-118.59062326 -136.45000285  146.18117422]
**** Processing seed 8 : [ -99.00937295 -115.36250252  158.23117441]
**** Processing seed 9 : [ -97.50312293 -189.9218787    81.41242319]
**** Processing seed 10 : [ -89.97187281 -147.74687803   46.76867265]
**** Processing seed 11 : [ -47.79687214 -126.65937769   27.18742234]
**** Processing seed 12 : [ -74.15624756 -110.09062743  140.90929914]
**** Processing seed 13 : [ -86.20624775 -135.69687784   28.69367236]
**** Processing seed 14 : [-10

In [21]:
imMath.SetInput(tubeMaskImage)
imMath.AddImages(imLungs, 200, 1)
blendIm = imMath.GetOutput()

itk.imwrite(blendIm, "results/CT-Lungs-InitVess.mha")

In [22]:
LabelMapType = itk.Image[itk.UC,3]

trMask = ttk.ComputeTrainingMask[ImageType,LabelMapType].New()
trMask.SetInput( tubeMaskImage )
trMask.SetGap( 4 )
trMask.SetObjectWidth( 1 )
trMask.SetNotObjectWidth( 1 )
trMask.Update()
fgMask = trMask.GetOutput()

In [24]:
itk.imwrite(fgMask, "results/CT-Lungs-")

In [25]:
enhancer = ttk.EnhanceTubesUsingDiscriminantAnalysis[ImageType,LabelMapType].New()
enhancer.AddInput( imLungs )
enhancer.SetLabelMap( fgMask )
enhancer.SetRidgeId( 255 )
enhancer.SetBackgroundId( 128 )
enhancer.SetUnknownId( 0 )
enhancer.SetTrainClassifier(True)
enhancer.SetUseIntensityOnly(True)
enhancer.SetScales([0.43,1.29,3.01])
enhancer.Update()
enhancer.ClassifyImages()

In [26]:
im1vess = itk.SubtractImageFilter( Input1=enhancer.GetClassProbabilityImage(0), Input2=enhancer.GetClassProbabilityImage(1))

imMath.SetInput(imLungs)
imMath.Threshold(-1,0.00001,0,1)
imMath.Erode(8,1,0)
imMaskedE = imMath.GetOutput()

imMath.SetInput(im1vess)
imMath.ReplaceValuesOutsideMaskRange(imMaskedE, 1, 1, 0)
im1vessMasked = imMath.GetOutput()

In [27]:
itk.imwrite( im1vess, "results/CT-Lungs-VesselEnhanced.mha", compression=True)

itk.imwrite( im1vessMasked, "results/CT-Lungs-Masked-VesselEnhanced.mha", compression=True)